In [2]:
import xarray as xr
import rioxarray
import ee
from pathlib import Path
import sys, shutil, os, subprocess
import pandas as pd
import geopandas as gpd
from google.cloud import storage

root = Path.cwd()

In [3]:
ee.Authenticate()
ee.Initialize(project='harvardforest')

In [ ]:
def save_ts_as_tifs(site_name,epsg,nc_filename,dates,band):

    ts = xr.open_dataarray(root / 'data' / site_name / f'{nc_filename}.nc')

    # add crs
    ts = ts.rio.write_crs(epsg).rio.set_spatial_dims(x_dim='x',y_dim='y').rio.write_coordinate_system()

    # convert time coords to datetime
    years = pd.to_datetime(dates,format='%m-%Y')
    ts = ts.assign_coords({'time':years})

    nodatavalue = -9999.0
    ts = ts.astype('float32').rio.write_nodata(nodatavalue)

    out_dir = root/ 'data' / site_name / 'tifs'
    out_dir.mkdir(exist_ok=True)

    tifs = []
    times = pd.to_datetime(ts.time.values)
    for time, dt in zip(ts.time.values, times):

        tif_path = out_dir / f'{band}_{dt:%m%Y}.tif'

        r = ts.sel(time=time,band=band)
        r = r.astype('float32').rio.write_nodata(-9999.0)

        r.rio.to_raster(tif_path.as_posix(),compress='LZW',predictor=2,tiled=True)

        tifs.append((tif_path,int(dt.timestamp()*1000)))
    
    return tifs

# recreate tifs list
def make_tifs_list(site_name,dates):
    tif_names = list(os.listdir(root / 'data' / site_name / 'tifs'))
    years = pd.to_datetime(dates,format='%m-%Y')
    y = [int(year.timestamp()*1000) for year in years]
    tifs = list(zip(tif_names,y))

    return tifs

## upload tifs to gs bucket
def upload_tifs_to_gs(root,tif_name,folder_name,gs_bucket):

   client = storage.Client()
   try:             
      bucket = client.get_bucket(gs_bucket)
   except Exception:
      bucket = client.create_bucket(gs_bucket, location='us-east1')
      print(f'creating {gs_bucket} bucket')

   blob = bucket.blob(tif_name)  #instantiates blob object for this bucket
    
   # Upload file to the blob
   local_path = root / 'data' / folder_name / 'tifs' / tif_name
   blob.upload_from_filename(str(local_path))
   print(f'uploaded {tif_name} to gs://{gs_bucket}/{tif_name}')

# remove gs bucket
def empty_gs_bucket(gs_bucket):
    client = storage.Client()
    bucket = client.bucket(gs_bucket) 
    
    for blob in bucket.list_blobs():  
        blob.delete() 
        print(f"Deleted {blob.name}")
    
    bucket.delete()
    print(f'deleted bucket named {gs_bucket}')

# upload from gs to gee
def add_tif_to_ee(tif_name,timestamp,project_root,collection_name,gs_bucket,nodatavalue=-9999.0):

    collection_id = f'{project_root}/{collection_name}'
    try: 
        ee.data.getAsset(collection_id)
        print('asset exists')
    except Exception:
        subprocess.run(['earthengine','create','collection',collection_id],check=True)
        print('created asset')

    asset_id = f'{collection_id}/{tif_name.split('.')[0]}'  # remove .tif extension to name images

    asset_path = f'gs://{gs_bucket}/{tif_name}'
    
    cmd = ["earthengine","upload","image",
           f'--asset_id={asset_id}',
           f'--time_start={timestamp}',
           f'--nodata_value={nodatavalue}',
           '--pyramiding_policy=MEAN',
           '--force',
           asset_path,
           '--wait']
    
    cp = subprocess.run(cmd,text=True, capture_output=True)
    if cp.returncode != 0:
        print("STDOUT:\n", cp.stdout)
        print("STDERR:\n", cp.stderr)

    subprocess.run(["earthengine","set",asset_id,f"parents={collection_id}"])

    return asset_id

In [ ]:
# tifs = save_ts_as_tifs(site_name='ordway',epsg=26917,nc_filename='ordway_summer_ts_mosaiced',dates=['07-2016','07-2017','07-2018','07-2019','07-2020','07-2021','07-2022','07-2023','07-2025'],band='evi')

tifs = make_tifs_list(site_name='ordway',dates=['07-2016','07-2017','07-2018','07-2019','07-2020','07-2021','07-2022','07-2023','07-2025'])

In [ ]:
for tif, _ in tifs:
    upload_tifs_to_gs(tif_name=tif, gs_bucket='ordway_ts')


uploaded_assets = []
for tif, timestamp in tifs:
    a = add_tif_to_ee(tif_name=tif,timestamp=timestamp,project_root='projects/harvardforest/assets',asset_name='evi_ts_ordway',gs_bucket='ordway_ts')
    uploaded_assets.append(a)
    
uploaded_assets

### not enough datapoints for landtrendr algorithm. need to use landsat instead.

['projects/harvardforest/assets/evi_ts_ordway/evi_072016',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072017',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072018',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072019',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072020',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072021',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072022',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072023',
 'projects/harvardforest/assets/evi_ts_ordway/evi_072025']

In [8]:
imcol = ee.ImageCollection('projects/harvardforest/assets/evi_ts_ordway').sort("system:time_start")

In [9]:
imcol.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'version': 1760397856358534,
 'id': 'projects/harvardforest/assets/evi_ts_ordway',
 'features': [{'type': 'Image',
   'bands': [{'id': 'b1',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [930, 782],
     'crs': 'EPSG:26917',
     'crs_transform': [10, 0, 399815, 0, -10, 3290125]}],
   'version': 1760397297408275,
   'id': 'projects/harvardforest/assets/evi_ts_ordway/evi_072016',
   'properties': {'system:time_start': 1467331200000,
    'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-82.03523967929058, 29.666683445356508],
      [-82.03522670835883, 29.66668284050803],
      [-81.93923826583583, 29.667397905018507],
      [-81.93919089220034, 29.667434799094192],
      [-81.93913860632415, 29.667466212931227],
      [-81.93913544252788, 29.667481036346324],
      [-81.93979151100882, 29.73796444920362],
      [-81.93983397208912, 29.738005657821862],
      [-81.93987018887323, 29.73805113736

In [ ]:
ee.Algorithms.TemporalSegmentation.LandTrendr(timeSeries, maxSegments, spikeThreshold, vertexCountOvershoot, preventOneYearRecovery, recoveryThreshold, pvalThreshold, bestModelProportion, minObservationsNeeded)